In [32]:
import numpy as np

In [42]:
...

array([-0.13782133])

In [22]:
a = True
not a

False

In [123]:
class Input:
    
    def __init__(self, input_shape, activation):
        self.input_shape = input_shape
        self.activation = activation
        
    def info(self):
        # we return name, dict. info
        return 'input', {'shape': self.input_shape[0], 'activation function': self.activation}
    
    def __str__(self):
        return self.info()

# or will Input inherit from Dense?
class Dense(Input):
    
    # let this be static
    hidden_num = 0
    
    def __init__(self, neurons, activation):
        self.neurons = neurons
        self.activation = activation
        Dense.hidden_num += 1
    
    def get_neurons(self):
        return self.neurons
    
    def get_activation(self):
        return self.activation
    
    #@staticmethod
    def info(self):
        # mimick Input
        return f'hidden {self.hidden_num}', {
            'shape': self.neurons, 'activation function': self.activation
        }
    
    def __repr__(self):
        return f"{(self.neurons, self.activation)}"
        
# how to make this a parent of Dense
class NeuralNet(Dense):
    
    def __init__(self, _type='artificial'):
        self.layers = dict()
        self.weights = np.array([])
        self.biases = np.array([])
        self.NODES = np.array([])  # len is simply number of neurons
        self.activation = list()
        self.input_shape = None
        self.output_shape = None
        
        self.w_visited = 0
        self.n_visited = 0
        
        self.curr_layer = 1
    
    # initialize weights and biases
    def add_layers(self, layers):
        self.input_shape = (layers[0].get_neurons(),)
        for L in range(1, len(layers)):
            # number of neurons for both layers
            lay1_n, activate = str.split(repr(layers[L-1]))
            lay1_n = eval(lay1_n[1:-1])
            activate = activate[1:-2]
            lay2_n = layers[L].get_neurons()
            curr_weights = np.random.randn(lay1_n*lay2_n,)
            curr_biases = np.random.randint(-10, 10, size=lay2_n)
            
            # adding weights of layer to array of weights
            self.weights = np.concatenate((self.weights, curr_weights))
            self.biases = np.concatenate((self.biases, curr_biases))
            self.activation.append(activate)
            
            # caching the information
            self.layers[f"layer #{L}"] = {
                'shape': (layers[L-1].get_neurons(),),
                'activation function': activate,
            }
        
        self.output_shape = (layers[-1].get_neurons(),)
        self.activation.append(layers[-1].get_activation())
        self.layers["output"] = {
            'shape': self.output_shape,
            'activation function': self.activation[-1]
        }
        
        self.NODES = np.random.randn(self.input_shape[0])
    
    def compile_model(self, loss, optimizer):
        pass
    
    # return table with all info; reference self.layers
    def info(self):
        return ""
    
    def sigmoid(self, z, derivative=False):
        if derivative:
            pass
        return 1 / (1 + np.e**-z)
    
    def activate(self, val, kind, forward=True):
        if kind.lower()=='relu':
            return max(0, val)
        elif kind.lower()=='sigmoid':
            return self.sigmoid(val, not forward)
        else:
            pass
            
    
    
    # x values at layer l
    def forward(self):
        # x_layer will be np.array([x1, x2, x3,..., xn]), where n is the number of neurons
        
        try:
            lay = self.curr_layer
            lays = list(self.layers.keys())
            shape = self.layers[lays[lay]]['shape'][0]
        except IndexError:
            print("Can't go forwards anymore")
        else:
            # number of weights needed for this forward pass
            num_w = self.layers[lays[lay-1]]['shape'][0] * shape
            W = self.weights[self.w_visited : self.w_visited+num_w]
            print(W)
            self.w_visited += num_w
            print(num_w)
            
            # storing values of subsequent nodes
            new_nodes = []
            for node in range(shape):
                # record number of previous layer's nodes
                n_prev = int(num_w/shape)
                # weighted sum of previous inputs plus bias
                next_node = np.sum(
                    W[node*n_prev : node*n_prev + n_prev] * self.NODES[self.n_visited : self.n_visited+n_prev] + 0
                )
                # transform in some activation function
                act_func = self.activation[lay]
                next_node = self.activate(next_node, kind=act_func, forward=True)
                new_nodes.append(next_node)
                
            self.n_visited += n_prev
            self.NODES = np.concatenate((self.NODES, np.array(new_nodes)))
            
        
        self.curr_layer += 1
    
    # start by specifying neuron values with x
    def fit(self, x, y, epochs=1, metrics=[], *args, **kwargs):
        assert type(x)==np.ndarray and type(y)==np.ndarray
        if x.shape != self.input_shape:
            raise ValueException("X shaped does not match that of input layer's")
        if y.shape != self.output_shape:
            raise ValueException("Y shaped does not match that of output layer's")
        
        
        #### Start with caching input values
        X = np.array([x[:self.input_shape]])
        
        print(X)
        
        
#         for epoch in range(epochs):
#             # compute forward(x[...], y[...], epoch)
            
#             pass
        
        


In [88]:
ann.layers

{'layer #1': {'shape': (60,), 'activation function': 'relu'},
 'layer #2': {'shape': (120,), 'activation function': 'relu'},
 'output': {'shape': (2,), 'activation function': 'sigmoid'}}

In [98]:
ann = NeuralNet()

In [99]:
layers = [Input(input_shape=(60,), activation='relu'),
          Dense(neurons=120, activation='relu'),
          Dense(neurons=40, activation='relu'),
          Dense(neurons=2, activation='sigmoid')]
ann.add_layers(
    layers
)

In [101]:
ann.NODES.shape

(60,)

In [102]:
ann.weights.shape

(12080,)

In [103]:
ann.biases

array([ -5.,   6.,   2.,   2.,  -3.,  -5.,  -8.,   1.,   5.,  -4.,   8.,
         9., -10., -10.,   4.,   3.,  -3.,  -8.,   6.,   9.,   7.,   3.,
       -10.,   5.,  -6.,  -7.,   0.,   0.,   2.,   5.,   5.,  -6.,  -7.,
        -9.,   0., -10.,   8., -10.,  -7.,   1.,  -5.,  -2.,  -4.,  -7.,
         2.,  -5.,   6.,   3.,  -9.,  -1.,   8.,  -5.,   9.,   6.,  -3.,
         2.,  -3.,   3.,  -7.,  -4.,  -8.,   7.,  -9.,  -2.,  -7.,   6.,
        -8.,   7.,   2.,   9.,  -2.,  -3.,  -6.,   8.,  -7.,  -3.,   3.,
         3.,  -1.,   7.,   7.,   7.,  -5.,  -3.,   1.,   8.,   4.,  -1.,
        -6.,   8.,   5.,   0.,  -3.,  -3.,  -3.,   7.,  -5.,  -6.,   9.,
         6.,   5., -10.,   7.,  -5.,   9.,   2.,   7.,   2.,   6.,   5.,
         3.,   7.,   8.,  -4.,   0.,   5.,   6.,  -4.,   3.,  -9.,  -8.,
        -7.,   2.,  -1.,  -9.,   4.,   9.,   8.,   0., -10.,  -3.,  -7.,
        -4.,  -3.,   6.,   5.,   2.,  -1.,   6.,   6.,   4.,   0.,  -9.,
         7.,  -7.,   4.,  -5.,  -1.,  -8.,   0.,   

In [110]:
ann.forward()

Can't go forwards anymore


In [109]:
ann.NODES.shape

(222,)

In [12]:
ann.forward()

[ 0.36352085 -2.06617258]
2


In [147]:
w_5 = np.array([0.3, 0.2, 0.9])
x_5 = np.array([1, `2, 3])
np.sum(w_5*x_5)

3.4000000000000004

In [193]:
ann.forward()

[-0.41724708  2.91896738]
2


In [194]:
ann.NODES

array([-0.12452795,  0.61054005, -0.6994094 ,  0.28667038, -1.16375719,
       -3.51658164])

In [101]:
ann.weights[:1]

array([0.80343933])

In [30]:
ann.biases

array([-4.,  5., -7.,  7.,  9., -2.,  2., -2.])

### Test this with actual data

In [2]:
from sklearn import datasets

In [7]:
X, y = datasets.fetch_openml(name="mnist_784", version=1,
                             return_X_y=True, as_frame=False)
X/=255.

In [ ]:
ann = NeuralNet()

ann.add_layers(
    [Dense(neurons=X[0].shape[0], activation='relu'),
    Dense(neurons=1000, activation='relu'),
    Dense(neurons=np.unique(y).shape[0], activation='sigmoid')]
)

In [ ]:
ann.compile_model(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Make sure shapes are good
ann.fit(
    X, y, metrics=['accuracy', 'loss'], epochs=10
)

In [ ]:
ann.predict(X[0])